# Multi-Agent Systems: Orchestrating Agents Through Graphs

Welcome to this multi-agent lab! Here we'll explore how to orchestrate multiple agents using a graph-based approach, allowing for more flexible and controlled agent interactions.

## Why Use Graph-Based Orchestration?
While individual specialized agents excel at specific tasks, the way they communicate and collaborate significantly impacts system performance. Graph-based orchestration provides precise control over agent interactions, allowing for more complex workflows and better resource management.

## What We'll Build
We'll create the same deep research agent system as before, but with an improved orchestration approach:

* Our supervisor will become an evaluator
* A Researcher agent that has access to the internet and can research topics
* A Writer agent that can take the information and create a report

This time, we'll enhance our implementation with:
* Reference passing between agents - allowing efficient sharing of information
* Structured controls to manage agent interactions - limiting how often agents can call each other
* Graph-based workflow patterns that enable more complex execution paths

At the end of this module, you'll have a more flexible "Deep Research" agentic system with better orchestration capabilities.

# Using the Tavily MCP Server
For the sake of variety, we'll use the Tavily MCP on our research agent to see how these pieces come together. We need the API key in order for it to work so lets import that

In [ ]:
import os 
from dotenv import load_dotenv
# Check if you created the .env file before running this notebook.
print('Does the .env file exist?', os.path.exists('.env'))
# from dotenv import load_dotenv
load_dotenv('.env')

TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

In [ ]:
# First we wrap our own MCP server in a MCPServerStdio object
from pydantic_ai.mcp import MCPServerStdio as PyAIMCPServerStdio

# You'll need to have NPM installed to run this command.
tavily_mcp_server = PyAIMCPServerStdio(  
    command = 'npx',
    args=['-y', 'tavily-mcp@0.1.4'],
    env= {
        'TAVILY_API_KEY': TAVILY_API_KEY
    }
)

In [ ]:
# Next lets create our researcher agent. 
from pydantic_ai import Agent as PyAIAgent
from agentic_platform.core.models.prompt_models import BasePrompt

RESEARCHER_SYSTEM_PROMPT = """
You are a specialized Research Agent with web search capabilities. Your role is to:

1. Analyze user queries and construct a question to query the internet with. 
2. Organize findings into comprehensive, well-sourced research briefs
3. Return the research brief in a well structured format that a writer can use to write a report.
4. Make sure to cite your sources with links in markdown format at the bottom of the research brief.

Provide only the research based of your web search results in a format that a writer can use to write a report.
Make sure to cite your sources with links in markdown format at the bottom of the research brief.
"""

USER_PROMPT = """
Here is the user query:
{user_query}

Please provide a research brief based on the user query.
"""

class ResearcherPrompt(BasePrompt):
    system_prompt: str = RESEARCHER_SYSTEM_PROMPT
    user_prompt: str = USER_PROMPT


# Add the MCP Server params to the agent.
researcher_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-5-haiku-20241022-v1:0',
    system_prompt=ResearcherPrompt().system_prompt,
    mcp_servers=[tavily_mcp_server]
)

In [ ]:
import nest_asyncio
import asyncio

# We need to apply the nest_asyncio to run the agent.
nest_asyncio.apply()

In [ ]:
# Test out the researcher agent. 
async def run_pydantic_ai_mcp_client(user_msg: str,agent: PyAIAgent):
    async with agent.run_mcp_servers():
        result = await agent.run(user_msg)
    return result

# Run the agent with a user message.
user_msg = 'Can you show me any weather alerts for California?'
researcher_results = asyncio.run(run_pydantic_ai_mcp_client(user_msg, researcher_agent))
print(researcher_results.data)

# Create Our Writer Agent
Next lets recreate our writer agent just like we did in the previous notebook

In [ ]:
# Create prompt using our BasePrompt class. 
WRITER_SYSTEM_PROMPT = """
You are a specialized Writer Agent responsible for crafting polished, cohesive reports from research provided by the Research Agent. Your role is to:

1. Transform the short research brief into a comprehensive report.
2. Organize information logically with appropriate sections and flow
3. Maintain a professional, authoritative tone appropriate for the subject matter
4. Ensure clarity, conciseness, and readability for the target audience

You will be provided with comprehensive research materials that include facts, figures, and sourced information. Your job is to synthesize this information without altering facts or adding unsupported claims.

Please use complete sentenences and paragraphs. No bullet points. Break up the report into section with headers with the following format:
Title:
[Title of the report]

Section 1:
[Section 1 of the report]

Conclusion:
[Conclusion of the report]
"""

WRITER_USER_PROMPT = """
Here is the research brief:
{research_brief}

Please write a report based on the research brief.
"""

class WriterPrompt(BasePrompt):
    system_prompt: str = WRITER_SYSTEM_PROMPT
    user_prompt: str = WRITER_USER_PROMPT

# Lets use a really fast model for the writer agent.
writer_agent: PyAIAgent = PyAIAgent(
    'bedrock:us.anthropic.claude-3-5-haiku-20241022-v1:0',
    system_prompt=WriterPrompt().system_prompt
)

# Create Our Evaluator(s)
Lastly we need to modify the supervisor agent to be more of an "evaluator". Because it's not entirely orchestrating these agents, it doesn't really need to be an agent. It can be simple evaluation prompt calling Bedrock to evaluate the research and then evaluate the writing

In [ ]:

# Prompt for evaluating research quality with chain of thought
RESEARCH_EVALUATOR_SYSTEM_PROMPT = """
You are a Research Evaluator responsible for assessing the quality and completeness of research results. Your role is to:

1. Analyze user queries to understand their information needs
2. Review research outputs for accuracy, thoroughness, and relevance
3. Ensure sources are properly documented and credible
4. Provide a clear assessment with a final decision: APPROVE or REVISE

First, think through your evaluation step by step within <thinking> tags. Consider these criteria:
- Accuracy: Does the research contain factual errors or misinterpretations?
- Completeness: Does it fully address all aspects of the user's query?
- Relevance: Is the information directly relevant to the query?
- Sources: Are sources properly documented and credible?

After your analysis, provide your final decision within <evaluation> tags, which must be ONLY either APPROVE or REVISE (no other text inside these tags).

If APPROVE, follow the tags with a brief justification.
If REVISE, follow the tags with specific improvement suggestions for the Research Agent.

Format your response as:
<thinking>
Your step-by-step analysis of the research...
</thinking>

<evaluation>APPROVE</evaluation> or <evaluation>REVISE</evaluation>

Followed by your justification or improvement suggestions.

Limit evaluation cycles to prevent excessive iterations - accept reasonable quality research rather than pursuing perfection.
"""

RESEARCH_EVALUATOR_USER_PROMPT = """
Please analyze the following research results for the query: {query}

RESEARCH CONTENT:
{research_content}

Provide your step-by-step evaluation using the thinking and evaluation tags.
"""

# Prompt for evaluating the final report with chain of thought
REPORT_EVALUATOR_SYSTEM_PROMPT = """
You are a Report Evaluator responsible for assessing the quality and effectiveness of final reports. Your role is to:

1. Compare the final report against the original research to ensure accuracy
2. Evaluate the report's structure, coherence, and readability
3. Assess whether the report effectively communicates the key findings
4. Provide a clear assessment with a final decision: APPROVE or REVISE

First, think through your evaluation step by step within <thinking> tags. Consider these criteria:
- Accuracy: Does the report faithfully represent the research without distortion?
- Completeness: Does it include all key information from the research?
- Coherence: Is the information organized logically with good flow?
- Readability: Is it written in clear, professional language appropriate for the audience?

After your analysis, provide your final decision within <evaluation> tags, which must be ONLY either APPROVE or REVISE (no other text inside these tags).

If APPROVE, follow the tags with a brief justification.
If REVISE, follow the tags with specific improvement suggestions for the Writer Agent.

Format your response as:
<thinking>
Your step-by-step analysis of the report compared to the research...
</thinking>

<evaluation>APPROVE</evaluation> or <evaluation>REVISE</evaluation>

Followed by your justification or improvement suggestions.

Limit evaluation cycles to prevent excessive iterations - accept reasonable quality reports rather than pursuing perfection.
"""

REPORT_EVALUATOR_USER_PROMPT = """
Please analyze the following report for the query: {query}

ORIGINAL RESEARCH:
{research_content}

FINAL REPORT:
{report_content}

Provide your step-by-step evaluation using the thinking and evaluation tags.
"""


class ResearchEvaluatorPrompt(BasePrompt):
    system_prompt: str = RESEARCH_EVALUATOR_SYSTEM_PROMPT
    user_prompt: str = RESEARCH_EVALUATOR_USER_PROMPT

class ReportEvaluatorPrompt(BasePrompt):
    system_prompt: str = REPORT_EVALUATOR_SYSTEM_PROMPT
    user_prompt: str = REPORT_EVALUATOR_USER_PROMPT

# Create the prompt instances
research_evaluator_prompt: BasePrompt = ResearchEvaluatorPrompt()
report_evaluator_prompt: BasePrompt = ReportEvaluatorPrompt()

# Write Helper Function
Lets write a helper function for our Graph that can generate structured from our prompts for our graph. This will give us our evaluation

In [ ]:
from agentic_platform.core.models.memory_models import Message, ToolCall
from agentic_platform.core.models.tool_models import ToolSpec
from agentic_platform.core.converter.llm_request_converters import ConverseRequestConverter
from agentic_platform.core.converter.llm_response_converters import ConverseResponseConverter
from agentic_platform.core.models.llm_models import LLMRequest, LLMResponse
from typing import Dict, Any
import boto3

from pydantic import BaseModel
from typing import Literal

# Helper function to call Bedrock. Passing around JSON is messy and error prone.
bedrock = boto3.client('bedrock-runtime')
def call_bedrock(request: LLMRequest) -> LLMResponse:
    kwargs: Dict[str, Any] = ConverseRequestConverter.convert_llm_request(request)
    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return ConverseResponseConverter.to_llm_response(converse_response)

class EvalResults(BaseModel):
    thinking: str
    evaluation: Literal["APPROVE", "REVISE"]

# This will force a tool call allowing us to get structured output.
def run_evaluation(prompt: BasePrompt) -> EvalResults:
    structured_output = ToolSpec(
        name="evaluate",
        description="Useful for thinking about and evaluating research brifs or research documents",
        model=EvalResults
    )

    request: LLMRequest = LLMRequest(
        system_prompt=prompt.system_prompt,
        messages=[Message(role="user", text=prompt.user_prompt)],
        model_id=prompt.model_id,
        hyperparams=prompt.hyperparams,
        tools=[structured_output],
        force_tool=structured_output.name
    )

    response: LLMResponse = call_bedrock(request)
    tool_invocation: ToolCall = response.tool_calls[0]
    return EvalResults.model_validate(tool_invocation.arguments)


In [ ]:
from typing import Dict

# Test the evaluator
inputs: Dict[str, str] = {
    "research_content": researcher_results.data,
    "query": user_msg
}
prompt: BasePrompt = ResearchEvaluatorPrompt(inputs=inputs)
run_evaluation(prompt)

# Construct Our Graph
If you remember from module2, we built a series of graphs for our workflow agents. We will be doing something similar here as well but with pydantic-graph. Pydantic Graph has no dependencies on PydanticAI and can be run standalone. It offers similar functionality as LangGraph. 

The benefit of Pydantic graphs is that they're type safe. The downside is that they're a bit more complicated.

## Pydantic Graph Crash Course / tl;dr
Pydantic-graph creates workflow graphs where nodes are Python dataclasses that inherit from BaseNode. Each node implements a run method that returns another node or an End object, creating a directed flow. The GraphRunContext maintains state throughout execution and can inject dependencies. Nodes use Python type hints to define valid transitions between states, making the workflow type-safe. The library supports visualizing graphs with Mermaid diagrams, can persist state between runs (allowing workflows to be paused and resumed), and works well with PydanticAI for GenAI applications, though it can be used independently. 

The full documentation can be found [here](https://ai.pydantic.dev/graph/#graph). 



In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Dict

from pydantic_graph import BaseNode, End, Graph, GraphRunContext, Edge

# State class to track the flow
@dataclass
class ResearchFlowState:
    user_query: str
    research_results: str = ""
    research_feedback: str = ""
    final_report: str = ""
    report_feedback: str = ""
    attempts: int = 0
    max_attempts: int = 3

# Graph nodes
@dataclass
class ResearchQuery(BaseNode[ResearchFlowState]):
    """Processes user query through Research Agent"""
    async def run(self, ctx: GraphRunContext[ResearchFlowState]) -> EvaluateResearch:
        print("Running Research Query")
        query = ctx.state.user_query
        if ctx.state.research_feedback:
            query = f"{query}\n\nPrevious feedback: {ctx.state.research_feedback}"
            
        # Run researcher agent
        async with researcher_agent.run_mcp_servers():
            result = await researcher_agent.run(query)
        
        ctx.state.research_results = result.data
        return EvaluateResearch()

@dataclass
class EvaluateResearch(BaseNode[ResearchFlowState]):
    """Evaluates research quality using direct LLM call"""
    async def run(self, ctx: GraphRunContext[ResearchFlowState]) -> WriteReport | ResearchQuery:
        print("Evaluating Research")
        ctx.state.attempts += 1

        inputs: Dict[str, str] = {
            "research_content": ctx.state.research_results,
            "query": ctx.state.user_query
        }

        # Run the evaluation
        prompt: BasePrompt = ResearchEvaluatorPrompt(inputs=inputs)
        feedback: EvalResults = run_evaluation(prompt)
        
        ctx.state.research_feedback = feedback.thinking
        
        # Simple decision logic
        if feedback.evaluation == "APPROVE" or ctx.state.attempts >= ctx.state.max_attempts:
            ctx.state.attempts = 0  # Reset for next phase
            return WriteReport()
        else:
            return ResearchQuery()

@dataclass
class WriteReport(BaseNode[ResearchFlowState]):
    """Creates report from research"""
    async def run(self, ctx: GraphRunContext[ResearchFlowState]) -> EvaluateReport:
        print("Writing Report")
        write_input = ctx.state.research_results
        if ctx.state.report_feedback:
            write_input = f"{write_input}\n\nPrevious feedback: {ctx.state.report_feedback}"
            
        # Call writer agent
        result = await writer_agent.run(write_input)
        ctx.state.final_report = result.data
        return EvaluateReport()

@dataclass
class EvaluateReport(BaseNode[ResearchFlowState]):
    """Evaluates final report using direct LLM call"""
    async def run(self, ctx: GraphRunContext[ResearchFlowState]) -> End[str] | WriteReport:
        print("Evaluating Report")
        ctx.state.attempts += 1
        
        inputs: Dict[str, str] = {
            "research_content": ctx.state.research_results,
            "query": ctx.state.user_query,
            "report_content": ctx.state.final_report
        }
        
        # Run the evaluation
        prompt: BasePrompt = ReportEvaluatorPrompt(inputs=inputs)
        feedback: EvalResults = run_evaluation(prompt)
        
        ctx.state.report_feedback = feedback.thinking
        
        # Simple decision logic
        if feedback.evaluation == "APPROVE" or ctx.state.attempts >= ctx.state.max_attempts:
            return End(ctx.state.final_report)
        else:
            return WriteReport()

In [ ]:
# Define the graph
research_graph = Graph(nodes=[ResearchQuery, EvaluateResearch, WriteReport, EvaluateReport])

# Function to run the graph
async def process_query(user_query, max_attempts=3):
    state = ResearchFlowState(user_query=user_query, max_attempts=max_attempts)
    result = await research_graph.run(ResearchQuery(), state=state)
    return result.output

# Sync wrapper
def run_query(user_query, max_attempts=3):
    return asyncio.run(process_query(user_query, max_attempts))

# Example usage
final_report = run_query("What are the differences between the US and EU around pasturization of milk? ")
print(final_report)

# Visualize Graph
Like LangGraph, we can visualize the graph using Mermaid diagrams. Lets take a look at what our graph looks like

In [ ]:
from IPython.display import Image, display

display(Image(research_graph.mermaid_image(start_node=ResearchQuery, direction="LR")))


# Conclusion
In this workshop we built a deep research agentic system composed of 2 agents and 2 structured output prompts using constructed using Pydantic graph. 

## Takeaways:
1. Graph frameworks like Pydantic or LangGraph are model / framework agnostic. They're just graphs. You can put any model from any provider in the graph so long as it returns the right type and context to the next node. 
2. Not everything is an agent problem. The evaluation prompts didn't need to be agents and it just created unecessary overhead. 
3. Graphs are great, but we also could have implemented this with simple python control flow. Graphs start to make sense when control flow get complicated and your code starts to look like spaghetti code.

This concludes the advanced agent concept workshop. In the next module we will start to focus more on running these agents in a production environment at scale.